https://velog.io/@changhtun1/Python-%EC%86%8C%EC%85%9C-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%9D%98-%EA%B0%90%EC%84%B1-%EC%98%88%EC%B8%A1

In [60]:
import pandas as pd
#warning 메시지 표시 안함
import warnings
warnings.filterwarnings(action = 'ignore')
from konlpy.tag import Okt # 형태소 분석에 사용할 konlpy 패키지의 Okt 클래스를 임포트하고 okt 객체를 생성
okt = Okt()

# Train 데이터 불러오기
train_df = pd.read_excel('./data/5movies.xlsx')

# 데이터 확인
print(train_df.head())

   Unnamed: 0  Unnamed: 1  score  \
0           1           0      1   
1           2           1      1   
2           3           2      1   
3           4           3      1   
4           5           4      1   

                                                text  
0  메시지는 차치하고 감각적으로(시각, 청각) 황홀했다. 영화는 가능하고, 소설은 불가...  
1  개봉일을 기다려, 모처럼 팔순의 친정어머니와 롯데 시네마에서 보았어요. 잔잔한 시냇...  
2  하....  이것이 제 심사평입니다. 이런 게 바로 영화 아니겠습니다. 오늘 하루 ...  
3  평양냉면 같은 영화. 막상 기대했다가 보면 이게뭐야? 할지도 모르지만 극장을 나선 ...  
4  이 영화자체가 미나리같다. 그리고 이 영화에 나오는 인물들도 미나리같다. 특별하고 ...  


In [61]:
# 댓글이 있는 항목만 담기(빈 댓글 삭제)
# text 컬럼이 non-null인 샘플만 train_df에 다시 저장
train_df = train_df[train_df['text'].notnull()]

# 수정된 train_df의 정보를 다시 확인
print(train_df.info())

# 분류 클래스의 구성을 확인
print(train_df['score'].value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14518 entries, 0 to 14517
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  14518 non-null  int64 
 1   Unnamed: 1  14518 non-null  int64 
 2   score       14518 non-null  int64 
 3   text        14518 non-null  object
dtypes: int64(3), object(1)
memory usage: 567.1+ KB
None
1    12056
0     2462
Name: score, dtype: int64


In [62]:
# 한글 외 문자 제거(옵션)
import re # 정규식을 사용하기 위해 re 모듈을 임포트
# ‘ㄱ ~‘힣’까지의 문자를 제외한 나머지는 공백으로 치환, 영문: a-z| A-Z
train_df['text'] = train_df['text'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
print(train_df.head())

# Train용 데이터셋의 정보를 재확인
print(train_df.info())
text = train_df['text'] # 시리즈 객체로 저장
score = train_df['score']

   Unnamed: 0  Unnamed: 1  score  \
0           1           0      1   
1           2           1      1   
2           3           2      1   
3           4           3      1   
4           5           4      1   

                                                text  
0  메시지는 차치하고 감각적으로 시각  청각  황홀했다  영화는 가능하고  소설은 불가...  
1  개봉일을 기다려  모처럼 팔순의 친정어머니와 롯데 시네마에서 보았어요  잔잔한 시냇...  
2  하   이것이 제 심사평입니다  이런 게 바로 영화 아니겠습니다  오늘 하루 하  할듯   
3  평양냉면 같은 영화  막상 기대했다가 보면 이게뭐야  할지도 모르지만 극장을 나선 ...  
4  이 영화자체가 미나리같다  그리고 이 영화에 나오는 인물들도 미나리같다  특별하고 ...  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 14518 entries, 0 to 14517
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  14518 non-null  int64 
 1   Unnamed: 1  14518 non-null  int64 
 2   score       14518 non-null  int64 
 3   text        14518 non-null  object
dtypes: int64(3), object(1)
memory usage: 567.1+ KB
None


In [63]:
# Train용 데이터셋과 Test용 데이터 셋 분리
# 1. 예측력을 높이기 위해 수집된 데이터를 학습용과 테스트 용으로 분리하여 진행
# 2. 보통 20~30%를 테스트용으로 분리해 두고 테스트

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(text, score , test_size=0.2, random_state=0)
print(len(train_x), len(train_y), len(test_x), len(test_y))

11614 11614 2904 2904


In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df=0.9)
tfv.fit(train_x)
tfv_train_x = tfv.transform(train_x)
print(tfv_train_x)

  (0, 8566)	0.4911095708539604
  (0, 5739)	0.44604272649494614
  (0, 943)	0.7482361094979268
  (1, 11344)	0.23482722728810002
  (1, 11342)	0.21609487448102846
  (1, 10945)	0.15924591737132837
  (1, 7708)	0.10594727196491936
  (1, 7052)	0.2424948067252637
  (1, 6929)	0.07333685185658732
  (1, 6889)	0.21894416790296278
  (1, 4795)	0.32370407281302543
  (1, 4794)	0.25351062424481946
  (1, 2974)	0.24546132381843289
  (1, 2610)	0.267498217593156
  (1, 1443)	0.3090780150651846
  (1, 1436)	0.15758500675289863
  (1, 1392)	0.25907119332940004
  (1, 1389)	0.23482722728810002
  (1, 787)	0.3090780150651846
  (1, 99)	0.31565477238663886
  (1, 72)	0.10395613733858862
  (2, 10335)	0.32009352258580004
  (2, 10334)	0.22408433455246593
  (2, 10059)	0.3282560126009281
  (2, 10003)	0.1438288535751151
  :	:
  (11613, 9288)	0.10399109025415759
  (11613, 8940)	0.2159028463112065
  (11613, 8936)	0.18654658127862705
  (11613, 8849)	0.2159028463112065
  (11613, 8647)	0.24469418132702525
  (11613, 8634)	0.118372

### LogisticRegression

In [65]:
from sklearn.linear_model import LogisticRegression # 이진 분류 알고리즘
from sklearn.model_selection import GridSearchCV # 하이퍼 파라미터 최적화

clf = LogisticRegression(random_state=0)
params = {'C': [15, 18, 19, 20, 22]}
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_train_x, train_y)

# 최적의 평가 파라미터는 grid_cv.best_estimator_에 저장됨
print(grid_cv.best_params_, grid_cv.best_score_)# 가장 적합한 파라메터, 최고 정확도 확인

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    2.7s finished


{'C': 18} 0.9133803497861118


In [66]:
tfv_test_x = tfv.transform(test_x)
# test_predict = grid_cv.best_estimator_.score(tfv_test_x,test_y)
test_predict = grid_cv.best_estimator_.predict(tfv_test_x)
from sklearn.metrics import accuracy_score
print('감성 분류 모델의 정확도 : ',round(accuracy_score(test_y, test_predict), 3))

감성 분류 모델의 정확도 :  0.918


### GaussianNB

In [67]:
from sklearn.naive_bayes import GaussianNB

GNB = GaussianNB()
GNB_params = {"var_smoothing":np.linspace(0.1, 1, 10)}
grid_cv_GNB = GridSearchCV(GNB, param_grid=GNB_params, cv=3, scoring='accuracy', verbose=1)
grid_cv_GNB.fit(tfv_train_x.todense(), train_y)

# 최적의 평가 파라미터는 grid_cv.best_estimator_에 저장됨
print(grid_cv_GNB.best_params_, grid_cv_GNB.best_score_)# 가장 적합한 파라메터, 최고 정확도 확인

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  1.3min finished


{'var_smoothing': 1.0} 0.9001206390601014


### SVM

In [68]:
from sklearn.svm import SVC

svc_model = SVC(random_state=1)
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
svc_params = [{'C': param_range, 
               'kernel': ['linear']},
              {'C': param_range, 
               'gamma': param_range, 
               'kernel': ['rbf']}]

svc_cv_GNB = GridSearchCV(svc_model, param_grid=svc_params, cv=3, scoring='accuracy', verbose=1)
svc_cv_GNB.fit(tfv_train_x, train_y)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 216 out of 216 | elapsed: 26.0min finished


GridSearchCV(cv=3, estimator=SVC(random_state=1),
             param_grid=[{'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                                1000.0],
                          'kernel': ['linear']},
                         {'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                                1000.0],
                          'gamma': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                                    1000.0],
                          'kernel': ['rbf']}],
             scoring='accuracy', verbose=1)

In [69]:
# 최적의 평가 파라미터는 grid_cv.best_estimator_에 저장됨
print(svc_cv_GNB.best_params_, svc_cv_GNB.best_score_)# 가장 적합한 파라메터, 최고 정확도 확인

{'C': 10.0, 'gamma': 0.1, 'kernel': 'rbf'} 0.9157915964795794


In [70]:
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

models = []
models.append(('LR', LogisticRegression(random_state=0), 
               {'C': [15, 18, 19, 20, 22]}))
models.append(('GNB', GaussianNB(), 
               {"var_smoothing":np.linspace(0.1, 1, 10)}))
models.append(('SVC', SVC(random_state=1), 
               {'C': param_range, 
               'gamma': param_range, 
               'kernel': ['rbf']}))

In [71]:
results_estimator = {}
results_score = {}
for name, model, param in models:
    print(name)
    gsearch = GridSearchCV(model, cv=3, param_grid=param, scoring='accuracy', verbose=1)
    if name == 'GNB':
        gsearch.fit(tfv_train_x.todense(), train_y)
    else:
        gsearch.fit(tfv_train_x, train_y)

    results_estimator[name]= gsearch.best_estimator_
    results_score[name]= gsearch.best_score_

LR
Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    2.9s finished


GNB
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  1.2min finished


SVC
Fitting 3 folds for each of 64 candidates, totalling 192 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 192 out of 192 | elapsed: 24.5min finished


In [43]:
best_model_name = max(results_score,key=results_score.get)
best_model = results_estimator[best_model_name]

In [44]:
import pickle
from datetime import datetime

filename = best_model_name + '_'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.pkl'
pickle.dump(best_model, open(filename,'wb'))

In [45]:
loaded_model = pickle.load(open(filename, 'rb'))

In [58]:
def load_model_predict(text):
    #입력 텍스트에 대한 전처리 수행
    input_text = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(text)
    input_text = [" ".join(input_text)]
    # 입력 텍스트의 피처 벡터화
    st_tfidf = tfv.transform(input_text)

    # 최적 감성 분석 모델에 적용하여 감성 분석 평가
    st_predict = loaded_model.predict(st_tfidf)

    #예측 결과 출력
    if(st_predict == 0):
        print(text, '\n>> 예측 결과: 부정 감성')
    else :
        print(text, '\n>> 예측 결과: 긍정 감성')

In [59]:
text_list = ['딱히 대단한 재미도 감동도 없는데 ~! 너무 과대 평가된 영화 중 하나',
                  '돈주고 본계 아깝다',
                  '이걸 왜 만든거야',
                  '과연 다시볼까 내가',
                  '정말 오랜만에 재미난 영화를 본듯',
                  '우리 시간 어쩔']
for text in text_list:
    load_model_predict(text)

딱히 대단한 재미도 감동도 없는데 ~! 너무 과대 평가된 영화 중 하나 
>> 예측 결과: 부정 감성
돈주고 본계 아깝다 
>> 예측 결과: 부정 감성
이걸 왜 만든거야 
>> 예측 결과: 부정 감성
과연 다시볼까 내가 
>> 예측 결과: 긍정 감성
정말 오랜만에 재미난 영화를 본듯 
>> 예측 결과: 긍정 감성
우리 시간 어쩔 
>> 예측 결과: 긍정 감성
